# Example: Optimal adversaries for dense MNIST model

This notebook gives an example where OMLT is used to find adversarial examples for a trained dense neural network. We follow the below steps:<br>
1.) A neural network with ReLU activation functions is trained to classify images from the MNIST dataset <br>
2.) OMLT is used to generate a mixed-integer encoding of the trained model using the big-M formulation <br>
3.) The model is optimized to find the maximum classification error (defined by an "adversarial" label) over a small input region <br>


## Library Setup
This notebook assumes you have a working PyTorch environment to train the neural network for classification. The neural network is then formulated in Pyomo using OMLT which therefore requires working Pyomo and OMLT installations.

The required Python libraries used this notebook are as follows: <br>
- `numpy`: used for manipulate input data <br>
- `torch`: the machine learning language we use to train our neural network
- `torchvision`: a package containing the MNIST dataset
- `pyomo`: the algebraic modeling language for Python, it is used to define the optimization model passed to the solver
- `onnx`: used to express trained neural network models
- `omlt`: the package this notebook demonstates. OMLT can formulate machine learning models (such as neural networks) within Pyomo

**NOTE:** This notebook also assumes you have a working MIP solver executable (e.g., CBC, Gurobi) to solve optimization problems in Pyomo. The open-source solver CBC is called by default.

In [1]:
# Import requisite packages
# data manipulation
import tempfile

import numpy as np

# pyomo for optimization
import pyomo.environ as pyo

# pytorch for training neural network
import torch
import torch.onnx
from torch import nn, optim
from torch.optim.lr_scheduler import StepLR
from torchvision import datasets, transforms

# omlt for interfacing our neural network with pyomo
from omlt import OmltBlock
from omlt.io.onnx import (
    load_onnx_neural_network_with_bounds,
    write_onnx_model_with_bounds,
)
from omlt.neuralnet import FullSpaceNNFormulation

2024-05-16 17:36:49.569530: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-16 17:36:49.599228: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Import the Data and Train a Neural Network

We begin by loading the MNIST dataset as `DataLoader` objects with pre-set training and testing batch sizes:


In [2]:
# set training and test batch sizes
train_kwargs = {"batch_size": 64}
test_kwargs = {"batch_size": 1000}

# build DataLoaders for training and test sets
dataset1 = datasets.MNIST(
    "../data", train=True, download=True, transform=transforms.ToTensor()
)
dataset2 = datasets.MNIST("../data", train=False, transform=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(dataset1, **train_kwargs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

Next, we define the structure of the dense neural network model:

In [3]:
hidden_size = 50


class Net(nn.Module):
    # define layers of neural network
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(784, hidden_size)
        self.hidden2 = nn.Linear(hidden_size, hidden_size)
        self.output = nn.Linear(hidden_size, 10)
        self.relu = nn.ReLU()
        self.softmax = nn.LogSoftmax(dim=1)

    # define forward pass of neural network
    def forward(self, x):
        x = self.hidden1(x)
        x = self.relu(x)
        x = self.hidden2(x)
        x = self.relu(x)
        x = self.output(x)
        return self.softmax(x)

We next define simple functions for training and testing the neural network:

In [4]:
# training function computes loss and its gradient on batch, and prints status after every 200 batches
def train(model, train_loader, optimizer, epoch):
    model.train()
    criterion = nn.NLLLoss()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data.view(-1, 28 * 28))
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 200 == 0:
            print(
                f"Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100.0 * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}"
            )


# testing function computes loss and prints overall model accuracy on test set
def test(model, test_loader):
    model.eval()
    criterion = nn.NLLLoss(reduction="sum")
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data.view(-1, 28 * 28))
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print(
        f"\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100.0 * correct / len(test_loader.dataset):.0f}%)\n"
    )

Finally, we train the neural network on the dataset.
Training here is performed using the `Adadelta` optimizer for five epochs.

In [5]:
# define model and optimizer
model = Net()
optimizer = optim.Adadelta(model.parameters(), lr=1)
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

# train neural network for five epochs
for epoch in range(5):
    train(model, train_loader, optimizer, epoch)
    test(model, test_loader)
    scheduler.step()

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.309185
Train Epoch: 0 [12800/60000 (21%)]	Loss: 0.233512
Train Epoch: 0 [25600/60000 (43%)]	Loss: 0.294385
Train Epoch: 0 [38400/60000 (64%)]	Loss: 0.198371
Train Epoch: 0 [51200/60000 (85%)]	Loss: 0.192688

Test set: Average loss: 0.1485, Accuracy: 9534/10000 (95%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.091085
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.186301
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.122492
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.110627
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.084353

Test set: Average loss: 0.1107, Accuracy: 9662/10000 (97%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.132310
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.084304
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.181169
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.031130
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.014465

Test set: Average loss: 0.1083, Accuracy: 9694/10000 (97%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.073255
Train Epoch: 3 [12800/60000 (21%)]	Lo

## Build a MIP Formulation for the Trained Neural Network

We are now ready to use OMLT to formulate the trained model within a Pyomo optimization model. We use a  binary variable representation to represent the ReLU activation logic which is created automatically using the `FullSpaceNNFormulation` object (one could also simply use the `ReluBigFormulation` object).

First, we define a neural network without the final `LogSoftmax` activation. Although this activation helps greatly in training the neural network model, it is not trivial to encode in the optimization model. The ranking of the output labels remains the same without the activation, so it can be omitted when finding optimal adversaries. 

In [6]:
class NoSoftmaxNet(nn.Module):
    # define layers of neural network
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(784, hidden_size)
        self.hidden2 = nn.Linear(hidden_size, hidden_size)
        self.output = nn.Linear(hidden_size, 10)
        self.relu = nn.ReLU()

    # define forward pass of neural network
    def forward(self, x):
        x = self.hidden1(x)
        x = self.relu(x)
        x = self.hidden2(x)
        x = self.relu(x)
        return self.output(x)


# create neural network without LogSoftmax and load parameters from existing model
model2 = NoSoftmaxNet()
model2.load_state_dict(model.state_dict())

<All keys matched successfully>

Next, we define an instance of the optimal adversary problem. We formulate the optimization problem as: <br>

$
\begin{align*} 
& \max_x \ y_k - y_j \\
& s.t. y_k = N_k(x) \\ 
&\quad |x - \bar{x}|_\infty \leq 0.05
\end{align*}
$

where $\bar{x}$ corresponds to an image in the test dataset with true label `j`, and $N_k(x)$ is the value of the neural network output corresponding to adversarial label `k` given input `x`. PyTorch needs to trace the model execution to export it to ONNX, so we also define a dummy input tensor `x_temp`.

In [7]:
# load image and true label from test set with index 'problem_index'
problem_index = 0
image = dataset2[problem_index][0].view(-1, 28 * 28).detach().numpy()
label = dataset2[problem_index][1]

# define input region defined by infinity norm
epsilon_infty = 5e-2
lb = np.maximum(0, image - epsilon_infty)
ub = np.minimum(1, image + epsilon_infty)

# save input bounds as dictionary
input_bounds = {}
for i in range(28 * 28):
    input_bounds[i] = (float(lb[0][i]), float(ub[0][i]))

# define dummy input tensor
x_temp = dataset2[problem_index][0].view(-1, 28 * 28)

We can now export the PyTorch model as an ONNX model and use `load_onnx_neural_network_with_bounds` to load it into OMLT.

In [8]:
with tempfile.NamedTemporaryFile(suffix=".onnx", delete=False) as f:
    # export neural network to ONNX
    torch.onnx.export(
        model2,
        x_temp,
        f,
        input_names=["input"],
        output_names=["output"],
        dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}},
    )
    # write ONNX model and its bounds using OMLT
    write_onnx_model_with_bounds(f.name, None, input_bounds)
    # load the network definition from the ONNX model
    network_definition = load_onnx_neural_network_with_bounds(f.name)

As a sanity check before creating the optimization model, we can print the properties of the neural network layers from `network_definition`. This allows us to check input/output sizes, as well as activation functions.

In [9]:
for layer_id, layer in enumerate(network_definition.layers):
    print(f"{layer_id}\t{layer}\t{layer.activation}")

0	InputLayer(input_size=[784], output_size=[784])	linear
1	DenseLayer(input_size=[784], output_size=[50])	relu
2	DenseLayer(input_size=[50], output_size=[50])	relu
3	DenseLayer(input_size=[50], output_size=[10])	linear


Finally, we can load `network_definition` as a full-space `FullSpaceNNFormulation` object.

In [10]:
formulation = FullSpaceNNFormulation(network_definition)

## Solve Optimal Adversary Problem in Pyomo

We now encode the trained neural network in a Pyomo model from the `FullSpaceNNFormulation` object. 

In [11]:
# create pyomo model
m = pyo.ConcreteModel()

# create an OMLT block for the neural network and build its formulation
m.nn = OmltBlock()
m.nn.build_formulation(formulation)

WARNING (W1002): Setting Var 'nn.scaled_inputs[202]' to a numeric value `0`
outside the bounds (0.27941176295280457, 0.3794117867946625).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'nn.scaled_inputs[203]' to a numeric value `0`
outside the bounds (0.6754902005195618, 0.7754902243614197).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'nn.scaled_inputs[204]' to a numeric value `0`
outside the bounds (0.5735294222831726, 0.6735294461250305).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'nn.scaled_inputs[205]' to a numeric value `0`
outside the bounds (0.5421568751335144, 0.6421568989753723).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'nn.scaled_inputs[206]' to a numeric value `0`
outside the bounds (0.18529412150382996, 0.2852941155433655).
    See also https://pyomo.readth

Next, we define an adversarial label as the true label plus one (or zero if the true label is nine), as well as the objective function for optimization.

In [12]:
adversary = (label + 1) % 10
m.obj = pyo.Objective(expr=(-(m.nn.outputs[adversary] - m.nn.outputs[label])))

Finally, we solve the optimal adversary problem using a mixed-integer solver.

In [13]:
pyo.SolverFactory("cbc").solve(m, tee=True)

Welcome to the CBC MILP Solver 
Version: 2.10.10 
Build Date: Apr 19 2023 

command line - /opt/conda/bin/cbc -printingOptions all -import /tmp/tmpoznymvux.pyomo.lp -stat=1 -solve -solu /tmp/tmpoznymvux.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 359 (-1749) rows, 1048 (-1644) columns and 38227 (-8079) elements
Statistics for presolved model
Original problem has 100 integers (100 of which binary)
Presolved problem has 74 integers (74 of which binary)
==== 998 zero objective 51 different
==== absolute objective values 51 different
==== for integers 74 zero objective 1 different
74 variables have objective of 0
==== for integers absolute objective values 1 different
74 variables have objective of 0
===== end objective counts


Problem has 359 rows, 1048 columns (50 with objective) and 38227 elements
Column breakdown:
0 of type 0.0->inf, 746 of type 0.0->up, 0 of type lo->inf, 
228 of type lo->up, 0 of type free, 0 of type fixed, 
0 of ty

{'Problem': [{'Name': 'unknown', 'Lower bound': 5.19746075, 'Upper bound': 5.19746075, 'Number of objectives': 1, 'Number of constraints': 359, 'Number of variables': 1048, 'Number of binary variables': 100, 'Number of integer variables': 100, 'Number of nonzeros': 50, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'User time': -1.0, 'System time': 42.27, 'Wallclock time': 43.87, 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 1107, 'Number of created subproblems': 1107}, 'Black box': {'Number of iterations': 152711}}, 'Error rc': 0, 'Time': 43.88876152038574}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}